In [8]:
import torch
from torch import nn
from torch.nn import functional as F

In [9]:
X = torch.rand(2, 20)

In [10]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [11]:
net = MLP()
net(X)

tensor([[ 0.1168, -0.0016,  0.0384, -0.1051, -0.0563,  0.2187, -0.0808,  0.0776,
         -0.2451, -0.0225],
        [ 0.0602,  0.0649, -0.1306, -0.1743, -0.1351,  0.2531, -0.1819,  0.1293,
         -0.2266,  0.0297]], grad_fn=<AddmmBackward0>)

In [12]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [13]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1018,  0.0297,  0.0051, -0.0985,  0.0743, -0.1414, -0.0755,  0.0274,
         -0.0287,  0.0803],
        [ 0.2197,  0.0367, -0.0562, -0.1531,  0.1753, -0.1253, -0.1353,  0.1428,
          0.0806,  0.0550]], grad_fn=<AddmmBackward0>)

In [ ]:
# 可以定义自己的模型，想怎么构造就怎么构造

class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()